# Neural Nets

Welcome to the third tutorial!

As we promised last time, in this assigment you are not going to be bothered with implementation details or further enginiering work (well, sort of...). Rather, this time you will work with 'state-of-the-art' neural network library (well again, sort of...) called **Keras**. Your task will be to play around with some models -- trying to fit them to some data and fine-tune their parameters.

You may need a bit more computing power (or time...) for this assignment. Thanks to Google Colab this should not be such a big deal but still, expect to spend quite some time fiddling with this assignment. We also have a special *bonus task* for you. That's right, bonus task in a bonus assignment.

So lets start as usual with some imports (we highly recomend you just run this part of the code and do not modify it).

In [ ]:
from sklearn.datasets import make_circles, make_moons, make_blobs
from matplotlib import pyplot as plt
import numpy as np
import gzip
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras import backend as K


def get_data(shape='circles', n_samples=200, std=0.1):
    if (shape == 'sample'):
        X, y = make_blobs(n_samples, centers=2)
    elif(shape == 'circles'):
        X, y = make_circles(n_samples, noise=std)
    elif(shape == 'spiral'):
        X, y = make_moons(n_samples, noise=std)
    else:
        print("Unknow shape! Drawing random sample data.")
        X = np.random.rand(n_samples, 2) * std
        y = np.random.randint(0, 2, n_samples)
    return X, y


class PlotBuilder:

    def __init__(self, plot, n_samples, X, y):
        self.X = X
        self.y = y
        self.plot = plot
        x0_data = np.array(plot[0].get_xdata())
        y0_data = np.array(plot[0].get_ydata())
        x1_data = np.array(plot[1].get_xdata())
        y1_data = np.array(plot[1].get_ydata())

        self.class_0 = np.zeros((len(x0_data), 2))
        self.class_1 = np.zeros((len(x1_data), 2))

        self.class_0[:, 0] = x0_data
        self.class_0[:, 1] = y0_data

        self.class_1[:, 0] = x1_data
        self.class_1[:, 1] = y1_data

        plot[0].figure.canvas.mpl_connect('button_press_event', self)
        plot[1].figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        if event.inaxes != self.plot[0].axes:
            return

        if event.inaxes != self.plot[1].axes:
            return

        if event.button == 1:
            # right click
            self.class_0 = np.vstack((self.class_0,
                                      [event.xdata,
                                       event.ydata]))
        elif event.button == 3:
            # left click
            self.class_1 = np.vstack((self.class_1,
                                      [event.xdata,
                                       event.ydata]))

        self.X = np.vstack((self.class_0, self.class_1))
        self.y = np.array([0] * len(self.class_0) + [1] * len(self.class_1))
        self.plot[0].set_data(self.class_0[:, 0], self.class_0[:, 1])
        self.plot[1].set_data(self.class_1[:, 0], self.class_1[:, 1])
        self.plot[0].figure.canvas.draw()
        self.plot[1].figure.canvas.draw()


def show_plot(shape='cirles', n_samples=200, std=0.1):
    X, y = get_data(shape, n_samples, std)
    X_0 = X[y == 0]
    X_1 = X[y == 1]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('Generated data of "{}" shape'.format(shape))
    class_1, = ax.plot(X_0[:, 0], X_0[:, 1], 'go')
    class_2, = ax.plot(X_1[:, 0], X_1[:, 1], 'ro')
    p = PlotBuilder([class_1, class_2], n_samples, X, y)

    plt.show()
    return p.X, p.y, p


def load_mnist(kind):
    labels_path = './{}-labels-idx1-ubyte.gz'.format(kind)
    images_path = './{}-images-idx3-ubyte.gz'.format(kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)
    return images.reshape(len(images), 28, 28), labels


def visualize_mnist(X, y):
    label_idx = [y == i for i in range(10)]
    ims = np.array([X[label_idx[i]][:10] for i in range(10)])

    classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    f, axarr = plt.subplots(10, 10, figsize=(12, 12))
    for i in range(10):
        axarr[0][i].set_title(classes[i])
        for j in range(10):
            axarr[i][j].imshow(ims[j][i], 'gray')
            axarr[i][j].axis('off')

    plt.show()

# Download data into the notebook environment
!wget -q https://github.com/NaiveNeuron/ml_exercises/raw/master/assignment3/test-images-idx3-ubyte.gz
!wget -q https://github.com/NaiveNeuron/ml_exercises/raw/master/assignment3/train-images-idx3-ubyte.gz
!wget -q https://github.com/NaiveNeuron/ml_exercises/raw/master/assignment3/test-labels-idx1-ubyte.gz
!wget -q https://github.com/NaiveNeuron/ml_exercises/raw/master/assignment3/train-labels-idx1-ubyte.gz

# Ensure matplotlib images are shown
%matplotlib inline

Let's take a look at our data. We will start with what are commonly called *toy datasets*. They can be generated by the `show_plot()` function, which can take 4 different values as its first parameter.

* spiral
* circles
* sample
* anything else (this results in an unkown shape and random data)

In [ ]:
X, y, p = show_plot('spiral')

print("Starting shape of data - X: {} | y: {}".format(X.shape, y.shape))

Keras (https://keras.io/) is a high-level neural networks API, written in Python. As a backend it can use one of the more lower-level, 'research like' neural network libraries, such as, <a href='https://github.com/tensorflow/tensorflow'>Tensorflow</a> or <a href='https://github.com/Microsoft/cntk'>CNTK</a>. 

All of these libraries operate on the same principle. They build **computational graphs** which are then used to compute gradients using the backpropagation algorithm (if you are interested you can read more about computational graphs and backpropagation here: http://cs231n.github.io/optimization-2/).

**Keras** library was developed to be used as a quick prototyping tool. It allows you to specify your models and its parts as you would describe them on paper. It supports most of today's standard neural network building blocks as well as most of the deep learning models (Convolutional Neural Networks, Recurent Neural Networks, even [Transformers](https://keras.io/examples/nlp/text_classification_with_transformer/)...).

The core Keras datastructure is a model. The simplest available model is `Sequential()`, which is just a linear stack of layers (you can build more complex models through Keras functional API). This function returns a model datastructure, to which we can add layers using the `.add()` function.

The simplest model we can build is called a **Multi Layered Perceptron (MLP)**. This model contains just fully-connected layers. In Keras these are called **Dense** layers. 

The first parameter to the layer is number of neurons it has. We can then specify the **activation function** to be applied on its outputs by setting the `activation` parameter. First layer of any network also needs to have the `input_dim` parameter set. This tells Keras what is the input dimension of the data that is going to be fed into the model. This is needed so that Keras can prepare its weights.

We prepared a simple example model that consits of two layers. The first one has 6 neurons and uses the `tanh` activation function. The second (output) layer has two neurons (one for each class) and softmax activation function. Values at the last layer can now be represented as the probability that current example is part of one of the two classes.

After you specify your model, you need to compile it so that Keras can build its computational graph and prepare the weights. Each time you change your architecture you need to recompile your model (if you are not a fan of this, check out <a href='http://pytorch.org/'>PyTorch</a> or <a href='https://chainer.org/'>Chainer</a> which build their graphs on the fly).

Our targets are now in one dimensional array with values 0 and 1. This would be fine if our model would output just the probability value at the end (we could for instance use one sigmoid neuron as the output layer). But since we have two neurons with softmax activation (as we want to interpret this as probability of assigment to each of the classes) we need to transform our data into something called **categorical variable** by adding what is called *dummy variables*.

Each target variable now consits of 2 dimensions. The dimension that represents the *true* value in the previous one dimensional vector is set to **one**. Every other dimension si set to **zero**. Keras provides a handy function `to_categorical` that does this for us. Note that this process is sometimes also called one-hot encoding.

Feel free to check what this actually does in the next two cells.

In [ ]:
y[90:110]

In [ ]:
to_categorical(y[90:110])

Finally, we come to the last step: model training. This can be done using the `fit()` function. You will mostly need to provide a **loss** parameter (that is what loss is the model supposed to optimize), and you can also specify a set of metrics to be computed on the data by setting the **metrics** parameter (it expects a list of strings that represent respective metrics).

Optimizer is another required parameter that specifies the algorithm that your model is going to be optimized with. In our case we are going to use standard stochastic gradient descent (SGD). 

Keras also prepares the traning/validation set split by using the `validation_split` parameter. It takes float values from 0.0 to 1.0 which represent the fraction of the train set which is going to be used for validation. In our case it is 10%, meaning we set `validation_split=0.1`.

Let's try it all out in the next cell!

In [ ]:
mlp = Sequential()
mlp.add(Dense(6, activation='tanh', input_dim=X.shape[1]))
mlp.add(Dense(2, activation='softmax'))

mlp.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.4),  metrics=['accuracy'])

history = mlp.fit(p.X, to_categorical(p.y), epochs=30, validation_split=0.1, verbose=True)

Fit function returns a `history` object. This contains the history of all values of loss and metrics throughout the training. It is often very useful to plot these values, so that we can better understand what is our model doing (is the loss going down, are we overfitting/underfitting/...) throughout the training process.

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend(loc='best')

plt.figure()
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend(loc='best')
plt.show()

Keras also provides a handy function that alows us to eveluate our models on the metrics that we previously specified. For example:

In [ ]:
score = mlp.evaluate(p.X, to_categorical(p.y))
print("\n\nloss: {} | train acc: {}".format(score[0], score[1]))

We can also plot points that our model missclassified. 

Although this is just a toy example by far, error analysis like this is the key to getting intuition regarding both the problem and the model at hand, and at the same time improving the model/looking differently at the problem in the future.

In other words, unless you cannot for some reason, it is highly recommended to do it in any Machine Learning project you end up doing.

In [ ]:
Z = np.argmax(mlp.predict(p.X), axis=1)

wrong_points = p.X[Z != p.y]
wrong_classifications = p.y[Z != p.y]

plt.figure()

axes = plt.gca()
x_min, x_max = p.X[:, 0].min() - 1, p.X[:, 0].max() + 1
y_min, y_max = p.X[:, 1].min() - 1, p.X[:, 1].max() + 1
axes.set_xlim([x_min, x_max])
axes.set_ylim([y_min, y_max])

plt.scatter(wrong_points[:, 0], wrong_points[:, 1], c=wrong_classifications, cmap=plt.cm.coolwarm)
plt.title('Missclassified points')
plt.show()

For a simple classification task like this we can also plot something called *decission boundary*. It shows a threshold line through the *class space*. 

In [ ]:
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

Z = np.argmax(mlp.predict(np.c_[xx.ravel(), yy.ravel()]), axis=1)

Z = Z.reshape(xx.shape)

plt.figure()
plt.contourf(xx, yy, Z, alpha=0.6, cmap=plt.cm.coolwarm)

plt.scatter(p.X[:, 0], p.X[:, 1], c=p.y, cmap=plt.cm.coolwarm)
plt.title('Decision boundary')
plt.show()

Please, provide a brief explanation on what efect does changing the number of neurons or adding/subtracting layers have on the decision boundary and the resulting classification.

You can also try to change the learning rate or make use of different activation functions or optimizers (check the docs at https://keras.io/api/ for more details).

*Write your explanation here*

----------------------------------------------------------------------------------------------------------

## MNIST Fashion dataset

As the second part of this assigment, we prepared a new (also often times called *toy*) dataset called **MNIST Fashion dataset**. Note that both the name and the content of this dataset is a "play" on the (to some extend) overused "default" dataset in Machine Learning which for a long time used to be (and in some cases still is) the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database).


This (fashion) dataset consits out of 60 000 training examples and 10 000 test examples. Each datapoint is a grayscale picture that is 28x28 pixels in size. It consists out of 10 classes: 

* T-shirt/top
* Trouser
* Pullover
* Dress
* Coat
* Sandal
* Shirt
* Sneaker
* Bag
* Ankle boot

We already loaded this data for you so you can focus on optimizing your models. 

In [ ]:
X_train, y_train = load_mnist('train')
print("Train data shape: {} | labels: {}".format(X_train.shape, y_train.shape))

X_test, y_test = load_mnist('test')
print("Test data shape: {} | labels: {}".format(X_test.shape, y_test.shape))

In [ ]:
visualize_mnist(X_train, y_train)

First, just as before, let us start as before with a simple *MLP*. Since our data is in the shape  of a 28x28 pixel image, we need to create a single vector for each data point, so that it can be fed into our model.

*Note: Neural networks in general are very sensitive to input normalization. It is almost always a good idea to normalize your data. With images it can usualy be as simple as dividing any value by 255 which is the highest value that a pixel can have.*

In [ ]:
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)
print("New train data shape: {} | test data: {}".format(X_train.shape, X_test.shape))

In the cell below, there is a simple MLP with 3 layers prepared for you. The first two with 100 neurons each use the `tanh` activaton function. At the end, there is again a softmax activation function with 10 neurons (one for each class). 

Here we are going to be optimizing **categorical_crossentropy** with our new optimizer called **Adam** (https://arxiv.org/pdf/1412.6980.pdf, https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). We also prepared same code do display history objects.

In [ ]:
mlp = Sequential()
mlp.add(Dense(100, activation='tanh', input_dim=X_train.shape[1]))
mlp.add(Dense(100, activation='tanh'))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy', optimizer=Adam(0.04),  metrics=['accuracy'])

history = mlp.fit(X_train, to_categorical(y_train), epochs=30, validation_split=0.1, verbose=True)

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend(loc='best')

plt.figure()
plt.plot(history.history['acc'], label='train accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.legend(loc='best')
plt.show()

As you may recall, it is good practice to only compute test scores when you have already chosen the *best* model and only compute it **once**.

In [ ]:
train_score = mlp.evaluate(X_train, to_categorical(y_train))
print("\n\ntrain loss: {} | train acc: {}\n".format(train_score[0], train_score[1]))

test_score = mlp.evaluate(X_test, to_categorical(y_test))
print("\n\ntest loss: {} | test acc: {}".format(test_score[0], test_score[1]))

### Convolutional Neural Network

Now, we are going to try to use a **Convolutional Neural Network** on our data. First, some setup. Since convolutions can directly process 2D data (which is how our images are represented), we again reshape our data to 28x28x1 (based on the backend or settings you are using this can also be 1x28x28) -- the last digit is the number of channels our images have. 

Since we only have grayscale images, the number of channels is going to be 1. If we had full RGB images this value would be 3.

In [ ]:
img_rows = 28
img_cols = 28

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print("New train data shape: {} | test data: {}".format(X_train.shape, X_test.shape))

A convolution neural network is again just another `Sequential` model. We can add convolutional layers as before by running `.add(Conv2D())`, where the first parameter is the number of filters, the second is the size of these filters followed again by an activation function and for the first layer we also specify the input shape.

_**Note: for more information on all of these concepts, please consult the [Stanfornd's CS231n lecture notes](http://cs231n.github.io/convolutional-networks/) on the topic. Despite being a bit dated, they are still an excellent resource.**_

Once again, we prepared a simple starting model. It starts with two 2D convolutional layers, the first one having 32 filters and  the second one with 64 filters, each of them of size 3x3. After the first two layers, there is a MaxPoolling layer which just halved our output from the second convolutional layer by running max filter of size 2x2. At the end we have only one fully-connected layer with 10 neurons (again one for each class) with the softmax activation function.

This time we are going to run our optimizer on **batches** of data. This means that we are going to take a batch of 512 examples, compute the loss on all of these examples and backpropage afterwards. This can significantly speed up the process of training (at least "visually"), since we are not computing loss after every single example.


In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(10, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer=Adam(0.001),  metrics=['accuracy'])

history = cnn.fit(X_train, to_categorical(y_train), epochs=10, batch_size=512, 
                  validation_split=0.1, verbose=True)

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend(loc='best')

plt.figure()
plt.plot(history.history['acc'], label='train accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.legend(loc='best')
plt.show()

In [ ]:
X_train.shape

In [ ]:
train_score = cnn.evaluate(X_train, to_categorical(y_train))
print("\n\ntrain loss: {} | train acc: {}\n".format(train_score[0], train_score[1]))

test_score = cnn.evaluate(X_test, to_categorical(y_test))
print("\n\ntest loss: {} | test acc: {}".format(test_score[0], test_score[1]))

## For bonus points:

In this special bonus you are going to be competing against each other (and the outside world). We provided you with some starting code (also check out the first cell of this notebook) for the last dataset.

Now, **top solutions** on this Fashion MNIST dataset (in terms of **testing accuracy**) are going to be awarded bonus points. Furthermore, the **best report** (as evaluated by the TAs) will be awarded bonus points as well.

As your final submission, we would like to get the following from you: 

* This notebook, fully filled out.
* Everything necessary to get your results. **Your accuracy needs to be reproducible!** This means that we need to be able to get the same accuracy when we run your scripts.
* Max 2 page report (preferably in [Lecture Notes of Computer Science](https://github.com/latextemplates/LNCS) format) describing your approach and your results.

Some more notes:

* Your *testing accuracy* needs to be **at least 75%** in order for your solution to participate.
* Your models need to be neural-network based.
* Your models need to be trained on *just* the **training** data.
* Your models need to be implemented by you. If you are going to use a model that has already been described/used before, make sure you reference it in your report.

If you are stuck or would appreciate some help with this bonus, feel free to reach out to the [TAs](http://compbio.fmph.uniba.sk/vyuka/ml/).

Good luck!